# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    p_yes = (df[label]=='yes').mean()
    p_no = 1 - p_yes
    gini = 1-p_yes**2-p_no**2
    return gini    

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    values = df[attribute].unique()
    result = []
    for i in range(1, len(values)):
        result += [list(x) for x in combinations(values, i)]
    return result

In [6]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    splits = get_binary_split(df, attribute)
    result = {}
    for split in splits:
        pos = df[df[attribute].isin(split)]  # 카테고리에 해당되는 부분
        neg = df[~df[attribute].isin(split)]  # 아닌 부분
        gini = (len(pos) * get_gini(pos, label) + len(neg) * get_gini(neg, label)) / len(df)  # gini 값 계산
        result['_'.join(split)] = gini
    return result

In [8]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.39365079365079364,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.39365079365079364}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [9]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [10]:
##문제1 답안
get_binary_split(pd_data, 'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [11]:
def propose_feature(df, features):
    cands = []  # 각 피쳐별 최소 gini index를 가지는 카테고리가 이곳에 저장됨
    scores = []  # 그 때의 gini index가 이곳에 저장됨
    for feat in features:
        cand, score = min(get_attribute_gini_index(df, feat, "class_buys_computer").items())
        cands.append(cand)
        scores.append(score)
    index = np.argmin(scores) # 최소 gini index에 해당하는 인덱스
    return features[index], cands[index], scores[index]

In [12]:
##문제2 답안
left_features = [x for x in pd_data.columns if x != 'class_buys_computer']  # 탐색할 피쳐들 (타겟 제외)
propose_feature(pd_data, left_features)

('age', 'middle_aged', 0.35714285714285715)

In [13]:
##문제3 답안   
pos = pd_data[pd_data['age'] == 'middle_aged']
neg = pd_data[pd_data['age'] != 'middle_aged']

In [14]:
left_features = [x for x in left_features if x != 'age']  # 탐색할 피쳐들 (income 제외)
res = pd.DataFrame(columns=['feature', 'attribute', 'gini'])
res.loc['middle_aged'] = propose_feature(pos, left_features)
res.loc['not middle_aged'] = propose_feature(neg, left_features)
res

,feature,attribute,gini
middle_aged,income,high,0.00
not middle_aged,student,no,0.32
